In [1]:
pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11/11 [selenium]/11 [selenium]-client]
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install webdriver-manager

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [webdriver-manager]
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install --upgrade chromedriver-autoinstaller

Note: you may need to restart the kernel to use updated packages.


In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
from openpyxl import Workbook
import csv
import time

# 크롬 옵션 설정
options = Options()
options.add_argument("--start-maximized")
# options.add_argument("--headless")  # 창 없이 실행하고 싶다면 주석 해제

# 드라이버 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 유튜브 쇼츠 URL
url = "https://youtube.com/shorts/jKx_ZouWZLs?si=QLAGlu2Iaq0_a6wU"
driver.get(url)

# 명시적 대기 객체 생성
wait = WebDriverWait(driver, 10)

# 제품 정보 저장 리스트
product_data = []

def scroll_to_bottom():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # 페이지 로딩 시간
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

try:
    # "제품 보기" 버튼 클릭 (명시적 대기 사용)
    product_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "제품 보기")]')))
    product_button.click()
    print("✅ 제품 보기 버튼 클릭 완료")
    time.sleep(5)  # 제품 목록 로딩 대기

    # 페이지 끝까지 스크롤을 내려서 콘텐츠 로딩
    scroll_to_bottom()
    
    # 제품 목록 (명시적 대기 사용)
    products = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '[href*="shopping"]')))
    print(f"\n🔍 총 {len(products)}개의 제품 감지됨")

    for idx, product in enumerate(products, 1):
        name = "없음"
        price = "없음"
        brand_or_info = "없음"
        img_url = "없음"
        link = product.get_attribute("href")

        # 제품명
        if product.text:
            name = product.text.strip().split("\n")[0]

        # 이미지
        try:
            img = product.find_element(By.TAG_NAME, "img")
            img_url = img.get_attribute("src")
        except:
            pass

        # 추가 정보 (가격, 브랜드/기타)
        try:
            info_elements = product.find_elements(By.XPATH, ".//span")
            for elem in info_elements:
                text = elem.text.strip()
                if "₩" in text or "$" in text:
                    price = text
                elif text and text != name:
                    brand_or_info = text
        except:
            pass

        product_data.append([name, price, brand_or_info, img_url, link])

except Exception as e:
    print("❌ 오류 발생:", e)

driver.quit()

# -----------------------------------------
# 1. CSV 저장
csv_filename = f"shorts_products_{datetime.now():%Y%m%d}.csv"
with open(csv_filename, mode="w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["제품명", "가격", "브랜드/기타", "이미지 URL", "링크"])
    writer.writerows(product_data)
print(f"✅ CSV 파일 저장 완료: {csv_filename}")

# -----------------------------------------
# 2. Excel 저장
wb = Workbook()
ws = wb.active
ws.title = "YouTube Shorts 상품정보"
ws.append(["제품명", "가격", "브랜드/기타", "이미지 URL", "링크"])
for item in product_data:
    ws.append(item)

excel_filename = f"shorts_products_{datetime.now():%Y%m%d}.xlsx"
wb.save(excel_filename)
print(f"✅ 엑셀 파일 저장 완료: {excel_filename}")

# 모든 작업이 끝난 후 드라이버 종료
driver.quit()


❌ 오류 발생: Message: 
Stacktrace:
0   chromedriver                        0x00000001024e18b8 chromedriver + 5986488
1   chromedriver                        0x00000001024d89ea chromedriver + 5949930
2   chromedriver                        0x0000000101f91600 chromedriver + 415232
3   chromedriver                        0x0000000101fe3304 chromedriver + 750340
4   chromedriver                        0x0000000101fe3521 chromedriver + 750881
5   chromedriver                        0x0000000102033204 chromedriver + 1077764
6   chromedriver                        0x000000010200933d chromedriver + 906045
7   chromedriver                        0x0000000102030566 chromedriver + 1066342
8   chromedriver                        0x00000001020090e3 chromedriver + 905443
9   chromedriver                        0x0000000101fd561d chromedriver + 693789
10  chromedriver                        0x0000000101fd6281 chromedriver + 696961
11  chromedriver                        0x000000010249f580 chromedriver + 